In [ ]:
import tensorflow as tf
import os
import random
import numpy as np
import pandas as pd

from tqdm import tqdm 

from skimage.io import imread, imshow
from skimage.transform import resize
import matplotlib.pyplot as plt

In [ ]:
seed = 42
np.random.seed = seed

IMG_WIDTH = 64
IMG_HEIGHT = 64
IMG_CHANNELS = 3



In [ ]:
train = pd.read_csv("../input/petfinder-pawpularity-score/train.csv")
test = pd.read_csv("../input/petfinder-pawpularity-score/test.csv")
sample_submission = pd.read_csv("../input/petfinder-pawpularity-score/sample_submission.csv")

train["file_path"] = train["Id"].apply(lambda identifier: "../input/petfinder-pawpularity-score/train/" + identifier + ".jpg")
test["file_path"] = test["Id"].apply(lambda identifier: "../input/petfinder-pawpularity-score/test/" + identifier + ".jpg")

In [ ]:
g=[]
for n, id_ in tqdm(enumerate(train["file_path"]), total=len(train["file_path"])):
    image_string = tf.io.read_file(id_)
    image = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.central_crop(image, 1.0)
    image = tf.image.resize(image, (IMG_HEIGHT, IMG_WIDTH))
    g.append(image)


In [ ]:
r=[]
for n, id_ in tqdm(enumerate(test["file_path"]), total=len(test["file_path"])):
    image_string = tf.io.read_file(id_)
    image = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.central_crop(image, 1.0)
    image = tf.image.resize(image, (IMG_HEIGHT, IMG_WIDTH))
    r.append(image)


In [ ]:
imshow(r[5].numpy())
plt.show()

In [ ]:

imshow(g[5].numpy())
plt.show()

In [ ]:
def rmse(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean((y_true - y_pred) ** 2))

In [ ]:
def block(x, filters, kernel_size, repetitions, pool_size=2, strides=2):
    for i in range(repetitions):
        x = tf.keras.layers.Conv2D(filters, kernel_size, activation='relu', padding='same')(x)
    x = tf.keras.layers.MaxPooling2D(2, 2)(x)
    return x

In [ ]:

    early_stop = tf.keras.callbacks.EarlyStopping(
        monitor='val_rmse', 
        mode="min",
        min_delta=1e-4, 
        patience=10
    )
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_rmse', 
        mode="min",
        factor=0.5,
        patience=2, 
        min_lr=1e-6
    )
    callbacks = [early_stop, reduce_lr]
    
    loss = tf.keras.losses.MeanSquaredError()

    optimizer = tf.keras.optimizers.Adam()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from skimage import io

# Construct an instance of the ImageDataGenerator class
# Pass the augmentation parameters through the constructor. 

datagen = ImageDataGenerator(
        rotation_range=45,     #Random rotation between 0 and 45
        width_shift_range=0.2,   #% shift
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='constant', cval=125) 

In [ ]:
datagen.fit(g)

In [ ]:
from tensorflow import  keras
INPUT_SHAPE = (64, 64, 3)   
#models = []
#historys = []
#kfold = KFold(n_splits=5, shuffle=True, random_state=997)

image_inputs = tf.keras.Input(INPUT_SHAPE)

image_x = block(image_inputs, 8, 3, 2)
image_x = block(image_x, 16, 3, 2)
image_x = block(image_x, 32, 3, 2)
image_x = block(image_x, 64, 3, 2)
image_x = block(image_x, 128, 3, 2)
image_x = tf.keras.layers.Flatten()(image_x)
image_x = tf.keras.layers.Dense(64, activation="relu", kernel_regularizer=tf.keras.regularizers.l2())(image_x)


out = keras.layers.Dense(1)(image_x)   

model = keras.Model(inputs=image_inputs, outputs=out)



model.compile(optimizer=optimizer,
                loss=loss,   #Check between binary_crossentropy and categorical_crossentropy
                metrics=[tf.keras.metrics.RootMeanSquaredError(name="rmse"), "mae", "mape"])
print(model.summary())
    
    



In [ ]:
#for index, (train_indices, val_indices) in enumerate(kfold.split(g)):
 #   x_train = g.loc[train_indices]
    
  #  x_val= train.loc[val_indices]


In [ ]:
history = model.fit_generator(datagen.flow(np.array(g),train["Pawpularity"]),
                         
                         verbose = 1, 
                         epochs = 300,      #Changed to 3 from 50 for testing purposes.
                         
                         shuffle = True,
                         )


In [ ]:
history = model.fit(np.array(g), 
                         train["Pawpularity"], 
                         batch_size = 300, 
                         verbose = 1, 
                         epochs = 300,      #Changed to 3 from 50 for testing purposes.
                         validation_split = 0.20,
                         shuffle = True, 
                         callbacks=callbacks
                    
                      
                     )



In [ ]:
history = model.fit(np.array(g), 
                         train["Pawpularity"], 
                         batch_size = 300, 
                         verbose = 1, 
                         epochs = 300,      #Changed to 3 from 50 for testing purposes.
                         validation_split = 0.20,
                         shuffle = True, 
                         callbacks=callbacks
                    
                      
                     )

In [ ]:
model.save('malaria_cnn.h5')

In [ ]:
sample_submission["Pawpularity"]=model.predict(
    np.array(r),batch_size = 300, verbose = 1,  callbacks=callbacks
)

In [ ]:

sample_submission.to_csv("submission.csv", index=False)
sample_submission